# Channel Islands Reddit Analytics Engine Dashbord

In [46]:
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud, STOPWORDS
# from imgur import upload
import praw
import os
from ipywidgets import widgets
from IPython.display import display
import operator

text = widgets.Text(description ="Redditor:")
word_cloud_b = widgets.Button(description="Word Cloud")
karma_breakdown_b = widgets.Button(description="Karma Breakdown")
word_count_b = widgets.Button(description="Word Count")
user_activity_b = widgets.Button(description="User Activity")

user_agent = 'Reddit Analytics Engine, writted by CSUCI students'
reddit_client = praw.Reddit(user_agent=user_agent)

display(text)

def handle_submit(sender):
    display(word_cloud_b)
    display(karma_breakdown_b)
    word_cloud_b.on_click(word_cloud(text.value))
    karma_breakdown_b.on_click(karma_breakdown(text.value))
    
def word_cloud(target):
    print("Generating word_cloud, please wait...")

    # Set to grab a certain number of things from reddit...reddit wont return more than 1000
    thing_limit = 1000
    
    reddit_user = reddit_client.get_redditor(target)

    generated = reddit_user.get_comments(time='all', limit=None)

    # Stopwords from wordcloud
    s = set(STOPWORDS)
    s.add("one")
    
    total_comments = str()
    for thing in generated:
        total_comments += thing.body
        total_comments += " "

    # take relative word frequencies into account, lower max_font_size
    wordcloud = WordCloud(scale=3, relative_scaling=.5, random_state=1, stopwords=s).generate(total_comments)
    plt.figure()
    plt.title(reddit_user.name + "'s image cloud")
    plt.imshow(wordcloud)
    plt.axis("off")
    
    plt.show()
    
def karma_breakdown(target):
    print("Generating Karma Breakdown, please wait...")
    # Set to grab a certain number of things from reddit...reddit wont return more than 1000
    thing_limit = 100

    reddit_user = reddit_client.get_redditor(target)

    generated = reddit_user.get_submitted(limit=thing_limit)

    karma_by_subreddit = {}

    for thing in generated:
        subreddit = thing.subreddit.display_name
        karma_by_subreddit[subreddit] = (karma_by_subreddit.get(subreddit, 0) + thing.score)

    # remove entries that have a value of 0
    removed_zeroes = {x:y for x, y in karma_by_subreddit.items() if y != 0}

    #sorts the dictionary by value, then reverses
    sorted_x = sorted(removed_zeroes.items(), key=operator.itemgetter(1), reverse=True)

    #get just the keys into a sorted list, and trim to get the top 20
    sorted_keys = list(map(itemgetter(0), sorted_x))
    del sorted_keys[20:]

    #get just the keys into a sorted values, and trim to get the top 20
    sorted_values = list(map(itemgetter(1), sorted_x))
    del sorted_values[20:]

    max_y_tick = np.amax(sorted_values) + 1
    bar_width = 1
    sorted_range = np.array(range(len(sorted_keys)))

    fig, ax = plt.subplots()
    ax.bar(sorted_range, sorted_values, bar_width, align='center')

    ax.set_title("Karma Breakdown of top 20 subreddits from User: " + reddit_user.name)

    ax.set_xlabel("Subreddit Name")
    ax.set_ylabel("Karma", rotation='vertical')

    ax.set_xlim([0, len(sorted_range)])
    ax.set_ylim([0, max_y_tick])

    fig.tight_layout() # This is apparently supposed to give room to the x labels
    ax.axis('tight')

    ax.set_xticks(sorted_range)
    ax.set_yticks(range(max_y_tick))

    ax.set_xticklabels(sorted_keys, rotation='vertical')
    plt.show()

    
text.on_submit(handle_submit)
